Let $P_i$ be selling price for warehouse i $ \in $ {1,2,..,6} 

and let $L_i$ be lot size for warehouse i $ \in $ {1,2,..,6}

and let $E_i$ be elevation for warehouse i $ \in $ {1,2,..,6}

$ \tilde{P} \;  = b_0 + b_1 L_i + b_2 E_i \\ R_i = P_i - \tilde{P_i} \;  $

Objective min $  \sum_{i=1}^{6} |R_i|   $




Subjected to $R_i = P_i - \tilde{P_i} $

$ \tilde{P_i} = b_0 + b_1L_i + b_2E_i $


Objective: min $ \sum_{i=1}^{6} x_i $ 
subjected to :
$ x_i +b_0 + b_1L_i +b_2E_i \geq P_i \\ x_i -b_0 - b_1L_i -b_2E_i \geq -P_i \\ b_0 \geq 0$


In [1]:
!pip install -q pyomo 

     |████████████████████████████████| 9.1 MB 5.0 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np 

In [4]:
import pandas as pd

In [5]:
data_csvfile = pd.read_csv('lab6_ex3.csv')

In [6]:
data_csvfile.columns

Index(['Warehouse', 'Selling Price', 'Lot size', 'Elevation'], dtype='object')

In [7]:
model1 = ConcreteModel()

In [8]:
N = len(data_csvfile.index)
print(N)

6


In [9]:
M = 6

In [10]:
col_indices = range(N)

In [11]:
model1.x = Var(col_indices)

In [12]:
model1.b0 = Var(domain = NonNegativeReals)

In [13]:
model1.b1 = Var(domain = Reals)

In [14]:
model1.b2 = Var(domain = Reals)

In [15]:
model1.objective = Objective(expr = sum(model1.x[j] for j in col_indices), sense = minimize )

In [16]:
model1.constraints = ConstraintList()

In [17]:
for i in col_indices:
  model1.constraints.add(expr = model1.x[i] - model1.b0 - data_csvfile['Lot size'][i]*model1.b1 - data_csvfile['Elevation'][i]*model1.b2 >= data_csvfile['Selling Price'][i])

In [18]:
for i in col_indices:
  model1.constraints.add(expr = model1.x[i] + model1.b0 + data_csvfile['Lot size'][i]*model1.b1 + data_csvfile['Elevation'][i]*model1.b2 >= -data_csvfile['Selling Price'][i])

In [19]:
model1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

4 Var Declarations
    b0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    b1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    b2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
  

In [20]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [21]:
opt_cbc = SolverFactory('cbc')

In [22]:
result1 = opt_cbc.solve(model1)

In [23]:
print('Solver status:', result1.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [24]:
print('\nObjective = ', model1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x',i,' = ', model1.x[i].value)

print('\nConstraints')
model1.constraints.display()


Objective =  278392.859

Decision Variables
x 0  =  81607.143
x 1  =  0.0
x 2  =  41250.0
x 3  =  53214.286
x 4  =  0.0
x 5  =  102321.43

Constraints
constraints : Size=12
    Key : Lower     : Body                : Upper
      1 :  175000.0 :        175000.00022 :  None
      2 :  150000.0 :        150000.00012 :  None
      3 :  125000.0 :        207500.00015 :  None
      4 :   90000.0 :   90000.00031999999 :  None
      5 :   80000.0 :         80000.00003 :  None
      6 :  120000.0 :        120000.00145 :  None
      7 : -175000.0 : -11785.714220000009 :  None
      8 : -150000.0 :       -150000.00012 :  None
      9 : -125000.0 :       -125000.00015 :  None
     10 :  -90000.0 :         16428.57168 :  None
     11 :  -80000.0 :        -80000.00003 :  None
     12 : -120000.0 :   84642.85854999999 :  None


In [25]:
print("b0 :", model1.b0.value)
print("b1 :", model1.b1.value)
print("b2 :", model1.b2.value)

b0 : 0.0
b1 : 7.1428571
b2 : -425.0


let $|P_i - \tilde{P} | = | a_i - b_i| = a_i + b_i \\ a_i >= 0 b_i >= 0$

Objective : $\sum_{i=1}^{6} (a_i + b_i)$
 subjected to :
 
 $ a_i - b_i + b_0 + b_1L_i + b_2E_i = P_i \\ a_i \geq 0 \\ b_i \geq 0 \\ b_0 \geq 0$


In [26]:
model2 = ConcreteModel()

In [27]:
N = 6 

In [28]:
col_indices = range(6)

In [29]:
model2.a = Var(col_indices, domain= NonNegativeReals)

In [30]:
model2.b = Var(col_indices, domain = NonNegativeReals)

In [31]:
model2.objective = Objective(expr = summation(model2.a) + summation(model2.b), sense = minimize)

In [32]:
model2.constraints = ConstraintList()

In [33]:
model2.b0 = Var(domain=NonNegativeReals)
model2.b1 = Var(domain=Reals)
model2.b2 = Var(domain=Reals)

In [34]:
for i in col_indices:
  model2.constraints.add(expr = model2.a[i] - model2.b[i]+ model2.b0 + data_csvfile['Lot size'][i]*model2.b1 + data_csvfile['Elevation'][i]*model2.b2 == data_csvfile['Selling Price'][i])

In [35]:
model2.pprint()

3 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

5 Var Declarations
    a : Size=6, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : Fals

In [36]:
result2 = opt_cbc.solve(model2)

In [37]:
print('Solver status:', result2.solver.status)
print('Solver termination condition:',result2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [38]:
print('\nObjective = ', model2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('a',i,'=', model2.a[i].value)
  print('b',i,'=', model2.b[i].value)

print('\nConstraints')
model2.constraints.display()


Objective =  125454.545

Decision Variables
a 0 = 38818.182
b 0 = 0.0
a 1 = 0.0
b 1 = 0.0
a 2 = 0.0
b 2 = 54272.727
a 3 = 0.0
b 3 = 32363.636
a 4 = 0.0
b 4 = 0.0
a 5 = 0.0
b 5 = 0.0

Constraints
constraints : Size=6
    Key : Lower    : Body               : Upper
      1 : 175000.0 : 175000.00060000003 : 175000.0
      2 : 150000.0 :        149999.9996 : 150000.0
      3 : 125000.0 : 124999.99950000003 : 125000.0
      4 :  90000.0 :  90000.00160000002 :  90000.0
      5 :  80000.0 :  80000.00090000003 :  80000.0
      6 : 120000.0 : 120000.00150000001 : 120000.0


In [40]:
for i in col_indices:
  print('x',i,'=',model2.a[i].value-model2.b[i].value)

x 0 = 38818.182
x 1 = 0.0
x 2 = -54272.727
x 3 = -32363.636
x 4 = 0.0
x 5 = 0.0


In [41]:
print("b0 :", model2.b0.value)
print("b1 :", model2.b1.value)
print("b2 :", model2.b2.value)

b0 : 104181.82
b1 : 67.272727
b2 : -356.36364
